In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import numpy as np
from gensim.models import word2vec
from gensim.corpora.dictionary import Dictionary
import pickle
from keras.preprocessing import sequence

import time, datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import gc
from sklearn.model_selection import GridSearchCV
import jieba, pdb
from sklearn.decomposition import PCA


jieba.set_dictionary('jieba_dict/dict.txt.big')
# load stopwords set
stopword_set = set()
with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))

model = word2vec.Word2Vec.load("word2vec2.model")


def create_dictionaries(p_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(p_model.wv.vocab.keys(), allow_update=True)
    w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2indx.keys()}  # 词语的词向量
    return w2indx, w2vec

def Convert_orderid(x):
    return str(x).strip('\n')

def Convert_Date(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = pd.to_datetime(Year+'-'+Month+'-'+Day)
    return date1

def Date2Ticks(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = str(Year+'/'+Month+'/'+Day)
    return time.mktime(datetime.datetime.strptime(date1, "%Y/%m/%d").timetuple())

index_dict, word_vectors= create_dictionaries(model)
output = open("wordwmbedding.pkl", 'wb')
pickle.dump(index_dict, output)  # 索引字典
pickle.dump(word_vectors, output)  # 词向量字典
output.close()



C:\Users\Jess\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\Jess\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [2]:
df_train_1 = pd.read_csv("myTrain.csv")
df_result_1 = pd.read_csv("myTest.csv")


used_columns=['order_id','tick_diff', 'Source_1', 'Source_2', 'Unit', 'deal_or_not',
'people_amount', 'Begin_Tick','days', 'Order_Tick', 'Area', 'SubLine', 'price','PreDays','Begin_Date_Weekday', 
'Order_Date_Weekday', 'Return_Date_Weekday', 'fly_t', 'fly_date',
"src_airport", "arrive_t", "arrive_date", "dst_airport",'pred0','pred1','pred2','pred3','pred4','pred5','pred6', 'pred7','product_name']

df_train_1 = df_train_1[used_columns]
df_result_1 = df_result_1[used_columns]

myPred = []
for i in range(8):
    myPred.append(df_train_1['pred%d'%(i)].values.tolist())
myPred = np.asarray(myPred)
print(myPred.shape)
predFeat = []
for i in range(len(myPred[0,:])):
    predFeat.append([np.max(myPred[:,i]), np.percentile(myPred[:,i], 75), 
                     np.median(myPred[:,i]), np.percentile(myPred[:,i],25),np.min(myPred[:,i])]
                   )
train_feat=np.asarray(predFeat)

myPred = []
for i in range(8):
    myPred.append(df_result_1['pred%d'%(i)].values.tolist())
myPred = np.asarray(myPred)
predFeat = []
for i in range(len(myPred[0,:])):
    predFeat.append([np.max(myPred[:,i]), np.percentile(myPred[:,i], 75), 
                     np.median(myPred[:,i]), np.percentile(myPred[:,i],25),
                   np.min(myPred[:,i])])
test_feat=np.asarray(predFeat)

for i in range(8):
    del df_result_1['pred%d'%(i)]
    del df_train_1['pred%d'%(i)]

Y=df_train_1['deal_or_not'].values.tolist()
swX = (df_train_1['product_name']).values.tolist()
del df_train_1['deal_or_not'] 
del df_train_1['product_name']
del df_train_1['order_id']
X = df_train_1.values.tolist()

swrx = (df_result_1['product_name']).values.tolist()
rid = df_result_1['order_id'].values.tolist()
del df_result_1['product_name']
del df_result_1['deal_or_not'] 
del df_result_1['order_id'] 
rx = df_result_1.values.tolist()


(8, 296237)


In [3]:
sX, Y =np.asarray(X), np.asarray(Y)
X=np.concatenate([sX, train_feat], axis=1)
rx = np.asarray(rx)
rx=np.concatenate([rx, test_feat], axis=1)



def text_to_index_array(p_new_dic, p_sen):  # 文本转为索引数字模式
    new_sentences = []
    for sen in p_sen:
        new_sen = []
        for word in str(sen):
            try:
                new_sen.append(p_new_dic[word])  # 单词转索引数字
            except:
                new_sen.append(0)  # 索引字典里没有的词转为数字0
        new_sentences.append(new_sen)

    return np.array(new_sentences)


wX = text_to_index_array(index_dict, swX)
wrx = text_to_index_array(index_dict, swrx)

In [4]:
wX = sequence.pad_sequences(wX, maxlen=60)
wrx = sequence.pad_sequences(wrx, maxlen=60)


X=np.concatenate([X, wX], axis=1)
rx=np.concatenate([rx, wrx], axis=1)
# xlen=len(X)
# from sklearn.preprocessing import normalize
# Xtmp=normalize(np.concatenate([X, rx], axis=0),norm='max', axis=0)
# X=Xtmp[:xlen]
# rx=Xtmp[xlen:]

print(X.shape)
print(rx.shape)

print(X[5,-5:-1])

(296237, 86)
(99736, 86)
[8046. 8801.    0.    0.]


In [7]:

folds = StratifiedKFold(n_splits= 10, shuffle=True, random_state=666)

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(rx.shape[0])
feature_importance_df = pd.DataFrame()


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, Y)):

    train_x, train_y = X[train_idx,:], Y[train_idx]
    valid_x, valid_y = X[valid_idx,:], Y[valid_idx]

    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
    'nthread': 8, 'boosting_type': 'dart','objective': 'regression', 'metric': 'auc', 
        'learning_rate': 0.01, 'num_leaves': 70,
    'max_depth': 9, 'subsample': 0.7, 'feature_fraction': 0.7, 
        'min_split_gain': 0.09, 'min_child_weight': 9.5,
        'min_data_in_leaf':10,
        'bagging_freq':5,
    'drop_rate':0.5, 'skip_drop':0.5, 'max_drop':5, 'uniform_drop':False, 
        'xgboost_dart_mode':True, 'drop_seed':1 }


    if n_fold >= 0:
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dtrain,dval], early_stopping_rounds=200, verbose_eval=50)


        tmp_valid = bst.predict(valid_x, num_iteration=bst.best_iteration)
        oof_preds[valid_idx] = tmp_valid
        sub_preds += bst.predict(rx, num_iteration=bst.best_iteration) / folds.n_splits
        

        # Make the feature importance dataframe
        gain = bst.feature_importance('gain')
        fold_importance_df = pd.DataFrame({'feature':bst.feature_name(),'split':bst.feature_importance('split'),
                                           'gain':100*gain/gain.sum(),  'fold':n_fold,}).sort_values('gain',ascending=False)

        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

        del bst, train_x, train_y, valid_x, valid_y
        

Train Index: [     0      1      2 ... 296233 296234 296236] ,Val Index: [    15     37     46 ... 296213 296215 296235]
Training until validation scores don't improve for 200 rounds.
[50]	training's auc: 0.696865	valid_1's auc: 0.686925
[100]	training's auc: 0.698541	valid_1's auc: 0.687842
[150]	training's auc: 0.700427	valid_1's auc: 0.688887
[200]	training's auc: 0.702167	valid_1's auc: 0.689428
[250]	training's auc: 0.704289	valid_1's auc: 0.690127
[300]	training's auc: 0.706134	valid_1's auc: 0.690623
[350]	training's auc: 0.708071	valid_1's auc: 0.690983
[400]	training's auc: 0.710481	valid_1's auc: 0.691505
[450]	training's auc: 0.71275	valid_1's auc: 0.691899
[500]	training's auc: 0.714956	valid_1's auc: 0.692295
[550]	training's auc: 0.716929	valid_1's auc: 0.692714
[600]	training's auc: 0.719201	valid_1's auc: 0.693164
[650]	training's auc: 0.721466	valid_1's auc: 0.693517
[700]	training's auc: 0.723846	valid_1's auc: 0.693921
[750]	training's auc: 0.726028	valid_1's auc: 0.

[950]	training's auc: 0.733355	valid_1's auc: 0.702567
[1000]	training's auc: 0.735118	valid_1's auc: 0.702902
Did not meet early stopping. Best iteration is:
[1000]	training's auc: 0.735118	valid_1's auc: 0.702902
Fold  6 AUC : 0.702902
Train Index: [     0      1      2 ... 296234 296235 296236] ,Val Index: [    13     21     22 ... 296205 296209 296233]
Training until validation scores don't improve for 200 rounds.
[50]	training's auc: 0.695844	valid_1's auc: 0.691959
[100]	training's auc: 0.697711	valid_1's auc: 0.692837
[150]	training's auc: 0.699616	valid_1's auc: 0.693743
[200]	training's auc: 0.701456	valid_1's auc: 0.694289
[250]	training's auc: 0.703538	valid_1's auc: 0.695114
[300]	training's auc: 0.705529	valid_1's auc: 0.695665
[350]	training's auc: 0.707607	valid_1's auc: 0.696331
[400]	training's auc: 0.710187	valid_1's auc: 0.696921
[450]	training's auc: 0.712645	valid_1's auc: 0.69768
[500]	training's auc: 0.714671	valid_1's auc: 0.698025
[550]	training's auc: 0.716668

In [ ]:
def nanMapping(x):
    if np.isnan(x):
        return 0
    else:
        return x

app_test = pd.read_csv('testing-set.csv', usecols=['order_id'])
predF=pd.DataFrame({"order_id":rid, "deal_or_not":sub_preds})
preds = pd.DataFrame({"order_id":app_test['order_id']})
preds=preds.merge(predF, on="order_id",how="outer")
preds['deal_or_not'] = preds.deal_or_not.apply(lambda x: nanMapping(x))
preds.to_csv("output/lgb_dart_" + str(roc_auc_score(Y, oof_preds)) + ".csv", index=False)

In [ ]:
from sklearn.linear_model import Lasso, Ridge, RidgeCV, ElasticNet
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from catboost import Pool, CatBoostRegressor, cv


len1 = len(Y)
tind = np.zeros(len1, np.int)
for i in range(len1):
    tind[i]=i
import random as rn
rn.Random(4).shuffle(tind)
print(train_x.shape)

train_x, train_y = X[tind[1000:],:], Y[tind[1000:]]
valid_x, valid_y = X[tind[:1000],:], Y[tind[:1000]]

model_ridge = Ridge(alpha = 5)
model_ridge.fit(train_x, train_y)

model_lasso = Lasso(alpha = 0.005)
model_lasso.fit(train_x, train_y)

model_en = ElasticNet(alpha = 0.005)
model_en.fit(train_x, train_y)

model_gbr = GradientBoostingRegressor(n_estimators=1200, 
                                      learning_rate=0.05,
                                      max_depth=4, 
                                      max_features='sqrt',
                                      min_samples_leaf=15, 
                                      min_samples_split=10, 
                                      loss='huber',
                                      random_state=5)
model_gbr.fit(train_x, train_y)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.2,
                             learning_rate=0.06,
                             max_depth=3,
                             n_estimators=1150)
model_xgb(train_x, train_y)


params = {
'nthread': 8, 'boosting_type': 'dart','objective': 'regression', 'metric': 'auc', 
'learning_rate': 0.01, 'num_leaves': 70,
'max_depth': 9, 'subsample': 0.9, 'feature_fraction': 0.9, 
'min_split_gain': 0.09, 'min_child_weight': 9.5,
'drop_rate':0.5, 'skip_drop':0.5, 'max_drop':5, 'uniform_drop':False, 
'xgboost_dart_mode':True, 'drop_seed':5 }

dtrain = lgb.Dataset(train_x, label=train_y)
dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
bst = lgb.train(params, dtrain, num_boost_round=15000, valid_sets=[dval], early_stopping_rounds=2500, 
                verbose_eval=500)

tmp_valid = bst.predict(valid_x, num_iteration=bst.best_iteration)
sub_preds = bst.predict(rx, num_iteration=bst.best_iteration) 

# Make the feature importance dataframe
print('AUC : %.6f' % (roc_auc_score(valid_y, tmp_valid)))
def nanMapping(x):
    if np.isnan(x):
        return 0
    else:
        return x

app_test = pd.read_csv('testing-set.csv', usecols=['order_id'])
predF=pd.DataFrame({"order_id":rid, "deal_or_not":sub_preds})
preds = pd.DataFrame({"order_id":app_test['order_id']})
preds=preds.merge(predF, on="order_id",how="outer")
preds['deal_or_not'] = preds.deal_or_not.apply(lambda x: nanMapping(x))
preds.to_csv("output/lgb_dart_" + str(roc_auc_score(valid_y, tmp_valid)) + ".csv", index=False)

In [ ]:
preds['deal_or_not']